<a href="https://colab.research.google.com/github/lalith0203/Sales-Forecasting-using-LSTM/blob/main/Sales_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
dataset_path = 'RSCCASN.csv'

In [ ]:
sales_data = pd.read_csv(dataset_path,  parse_dates=True, index_col='DATE')

In [ ]:
type(sales_data)

In [ ]:
sales_data.columns

In [ ]:
sales_data.columns

In [ ]:
print(sales_data.info())

In [ ]:
print(sales_data.describe())

In [ ]:
sales_data.head()

In [ ]:
sales_data.tail()

In [ ]:
sales_data.plot(figsize=(10, 6))

In [ ]:
len(sales_data)

In [ ]:
len(sales_data) - 18

In [ ]:
test_size = 18
test_index = len(sales_data) - test_size

In [ ]:
test_index

In [ ]:
train = sales_data.iloc[:test_index]
test = sales_data.iloc[test_index:]

In [ ]:
train.head()

In [ ]:
train.tail()

In [ ]:
test.head()

In [ ]:
test.tail()

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(train)

In [ ]:
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [ ]:
scaled_train[:5]

In [ ]:
scaled_test[:5]

In [ ]:
length = 12

In [ ]:
generator = TimeseriesGenerator(scaled_train, scaled_train, length=length, batch_size=1)

In [ ]:
X, y = generator[0]

In [ ]:
X

In [ ]:
y

In [ ]:
len(X[0])

In [ ]:
n_features = 1

In [ ]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(length, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
validation_generator = TimeseriesGenerator(scaled_test, scaled_test, length=length, batch_size=1)

In [ ]:
model.fit(generator, epochs=70,
          validation_data=validation_generator,
          callbacks=[early_stop])

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses.plot()

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(len(test)):
    current_pred = model.predict(current_batch)[0]
    test_predictions.append(current_pred)
    current_batch = np.append(current_batch[:, 1:, :], [[current_pred]], axis=1)

In [ ]:
true_prediction = scaler.inverse_transform(test_predictions)

In [ ]:
test['Predictions'] = true_prediction

In [ ]:
test

In [ ]:
test.plot(figsize=(12, 8))

In [ ]:
full_scaler = MinMaxScaler()
scaled_full_data = full_scaler.fit_transform(sales_data)

In [ ]:
length = 12
generator = TimeseriesGenerator(scaled_full_data, scaled_full_data, length=length, batch_size=1)

In [ ]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(length, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(generator, epochs=20)

In [ ]:
forecast = []
periods = 12

first_eval_batch = scaled_full_data[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(periods):
    current_pred = model.predict(current_batch)[0]
    forecast.append(current_pred)
    current_batch = np.append(current_batch[:, 1:, :], [[current_pred]], axis=1)

In [ ]:
forecast = scaler.inverse_transform(forecast)

In [ ]:
sales_data

In [ ]:
forecast

In [ ]:
forecast_index = pd.date_range(start='2019-11-01', periods=periods, freq='MS')

In [ ]:
forecast_index

In [ ]:
forecast_data = pd.DataFrame(data=forecast, index=forecast_index, columns=['Forecast'])

In [ ]:
forecast_data